In [1]:
from xgboost import XGBClassifier
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from tqdm import tqdm
from scipy import sparse 
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import chi2, SelectPercentile,SelectKBest
import gc

In [2]:
from scipy.sparse import csr_matrix
def save_csr(matrix,file_prefix='csr'):
    data_file = file_prefix + '_data.npy'
    indices_file = file_prefix + '_indices.npy'
    indptr_file = file_prefix + '_indptr.npy'
    np.save(data_file,matrix.data)
    print('data save')
    np.save(indices_file,matrix.indices)
    print('indices save')
    np.save(indptr_file,matrix.indptr)
    print('indptr save')
    print('save done!')
    
def load_csr(file_prefix='csr'):
    data_file = file_prefix + '_data.npy'
    indices_file = file_prefix + '_indices.npy'
    indptr_file = file_prefix + '_indptr.npy'
    data = np.load(data_file)
    print('data load')
    indices = np.load(indices_file)
    print('indices load')
    indptr = np.load(indptr_file)
    print('indptr load')
    csr = csr_matrix((data, indices, indptr))
    return csr

In [3]:
train_csr = load_csr('../../data/features/xgb/train_csr_f620_b70_col17k')
predict_csr = load_csr('../../data/features/xgb/predict_csr_f620_b70_col17k')
train_y = np.load('../../data/features/xgb/train_y_f620_b70_col17k.npy')
predicted_file = pd.read_csv('../../data/features/xgb/predict_f620_b70_col17k.csv')

data load
indices load
indptr load
data load
indices load
indptr load


In [4]:
colnum = 17000
feature_select = SelectKBest(chi2,k=colnum)
feature_select.fit(train_csr, train_y)
train_csr = feature_select.transform(train_csr)
predict_csr = feature_select.transform(predict_csr)
print('feature select')
print(train_csr.shape)

feature select
(2992639, 17000)


In [5]:
leng = train_csr.shape[0]
train_csr_f = train_csr[:leng]
train_y_f = train_y[:leng]
predict_csr_f = predict_csr[:leng]
predict_result = predicted_file[['instance_id']].copy()
predict_result['predicted_score'] = 0

# define xgb

In [8]:
lgb_model = XGBClassifier(
    booster ='gbtree',
    objective='binary:logistic',
    reg_alpha=0, 
    reg_lambda=0.1,
#     max_depth=7, 
    n_estimators =50000, 
    subsample=0.7, 
    colsample_bytree=0.5,
    subsample_freq=1,
    learning_rate=0.1,
    random_state=1001, 
    nthread=16,
#     early_stopping_rounds = 250
)


In [9]:
n_fold = 5
gc.collect()
skf = StratifiedKFold(n_splits=n_fold, random_state=1001, shuffle=True)
best_score = []
oof = np.zeros(len(train_y_f))
for index, (train_index, test_index) in enumerate(skf.split(train_csr_f, train_y_f)):
    lgb_model.fit(train_csr_f[train_index], train_y[train_index],
                  eval_set=[(train_csr_f[train_index], train_y_f[train_index]),
                            (train_csr_f[test_index], train_y_f[test_index])], eval_metric =['logloss'],
                  early_stopping_rounds=250,verbose=10)
    best_score.append(lgb_model.evals_result_['validation_1']['logloss'][-1])
    print(best_score)
    print('cv:',np.mean(best_score))
    test_pred = lgb_model.predict_proba(predict_csr_f, ntree_limit=lgb_model.best_iteration)[:, 1]
    print('test mean:', test_pred.mean())
    predict_result['predicted_score'] = predict_result['predicted_score'] + test_pred
    oof[test_index] = lgb_model.predict_proba(train_csr_f[test_index], ntree_limit=lgb_model.best_iteration)[:, 1]
    gc.collect()

predict_result['predicted_score'] = predict_result['predicted_score']/n_fold

[0]	validation_0-logloss:0.652647	validation_1-logloss:0.652673
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 250 rounds.
[10]	validation_0-logloss:0.480956	validation_1-logloss:0.481082
[20]	validation_0-logloss:0.442929	validation_1-logloss:0.443116
[30]	validation_0-logloss:0.432534	validation_1-logloss:0.432744
[40]	validation_0-logloss:0.428903	validation_1-logloss:0.429156
[50]	validation_0-logloss:0.42723	validation_1-logloss:0.427501
[60]	validation_0-logloss:0.425921	validation_1-logloss:0.4262
[70]	validation_0-logloss:0.425016	validation_1-logloss:0.425311
[80]	validation_0-logloss:0.424302	validation_1-logloss:0.424614
[90]	validation_0-logloss:0.423692	validation_1-logloss:0.424027
[100]	validation_0-logloss:0.423164	validation_1-logloss:0.423506
[110]	validation_0-logloss:0.422733	validation_1-logloss:0.423118
[120]	validation_0-logloss:0.422362	validation_1-logloss

[1220]	validation_0-logloss:0.413721	validation_1-logloss:0.416355
[1230]	validation_0-logloss:0.413693	validation_1-logloss:0.416344
[1240]	validation_0-logloss:0.413662	validation_1-logloss:0.416332
[1250]	validation_0-logloss:0.413628	validation_1-logloss:0.416319
[1260]	validation_0-logloss:0.413598	validation_1-logloss:0.416307
[1270]	validation_0-logloss:0.413561	validation_1-logloss:0.41629
[1280]	validation_0-logloss:0.413534	validation_1-logloss:0.416281
[1290]	validation_0-logloss:0.4135	validation_1-logloss:0.416262
[1300]	validation_0-logloss:0.413473	validation_1-logloss:0.416249
[1310]	validation_0-logloss:0.413437	validation_1-logloss:0.416228
[1320]	validation_0-logloss:0.413405	validation_1-logloss:0.416217
[1330]	validation_0-logloss:0.413377	validation_1-logloss:0.416206
[1340]	validation_0-logloss:0.413346	validation_1-logloss:0.416195
[1350]	validation_0-logloss:0.413316	validation_1-logloss:0.416185
[1360]	validation_0-logloss:0.413289	validation_1-logloss:0.41617

[2450]	validation_0-logloss:0.410589	validation_1-logloss:0.41545
[2460]	validation_0-logloss:0.41057	validation_1-logloss:0.415443
[2470]	validation_0-logloss:0.410547	validation_1-logloss:0.41544
[2480]	validation_0-logloss:0.41053	validation_1-logloss:0.415443
[2490]	validation_0-logloss:0.410509	validation_1-logloss:0.415439
[2500]	validation_0-logloss:0.410485	validation_1-logloss:0.415434
[2510]	validation_0-logloss:0.410464	validation_1-logloss:0.415426
[2520]	validation_0-logloss:0.410447	validation_1-logloss:0.415422
[2530]	validation_0-logloss:0.410424	validation_1-logloss:0.415413
[2540]	validation_0-logloss:0.410402	validation_1-logloss:0.41541
[2550]	validation_0-logloss:0.410385	validation_1-logloss:0.415405
[2560]	validation_0-logloss:0.410369	validation_1-logloss:0.4154
[2570]	validation_0-logloss:0.410348	validation_1-logloss:0.415401
[2580]	validation_0-logloss:0.410326	validation_1-logloss:0.4154
[2590]	validation_0-logloss:0.410305	validation_1-logloss:0.415396
[260

[3680]	validation_0-logloss:0.408275	validation_1-logloss:0.415208
[3690]	validation_0-logloss:0.408257	validation_1-logloss:0.415206
[3700]	validation_0-logloss:0.40824	validation_1-logloss:0.415206
[3710]	validation_0-logloss:0.408221	validation_1-logloss:0.415207
[3720]	validation_0-logloss:0.408206	validation_1-logloss:0.4152
[3730]	validation_0-logloss:0.408188	validation_1-logloss:0.415199
[3740]	validation_0-logloss:0.408173	validation_1-logloss:0.415198
[3750]	validation_0-logloss:0.408155	validation_1-logloss:0.415197
[3760]	validation_0-logloss:0.408137	validation_1-logloss:0.415196
[3770]	validation_0-logloss:0.408121	validation_1-logloss:0.415196
[3780]	validation_0-logloss:0.408103	validation_1-logloss:0.415196
[3790]	validation_0-logloss:0.408086	validation_1-logloss:0.415195
[3800]	validation_0-logloss:0.40807	validation_1-logloss:0.415197
[3810]	validation_0-logloss:0.408052	validation_1-logloss:0.415201
[3820]	validation_0-logloss:0.408032	validation_1-logloss:0.415196

[610]	validation_0-logloss:0.416349	validation_1-logloss:0.417287
[620]	validation_0-logloss:0.416298	validation_1-logloss:0.417259
[630]	validation_0-logloss:0.416239	validation_1-logloss:0.41722
[640]	validation_0-logloss:0.41619	validation_1-logloss:0.417195
[650]	validation_0-logloss:0.416127	validation_1-logloss:0.41715
[660]	validation_0-logloss:0.416082	validation_1-logloss:0.41712
[670]	validation_0-logloss:0.41603	validation_1-logloss:0.417089
[680]	validation_0-logloss:0.41598	validation_1-logloss:0.417059
[690]	validation_0-logloss:0.415919	validation_1-logloss:0.417016
[700]	validation_0-logloss:0.415872	validation_1-logloss:0.41699
[710]	validation_0-logloss:0.415832	validation_1-logloss:0.416967
[720]	validation_0-logloss:0.415783	validation_1-logloss:0.416941
[730]	validation_0-logloss:0.415742	validation_1-logloss:0.416917
[740]	validation_0-logloss:0.415681	validation_1-logloss:0.416881
[750]	validation_0-logloss:0.415633	validation_1-logloss:0.416855
[760]	validation_

KeyboardInterrupt: 

In [ ]:
cv_loss = np.mean(best_score)
now = str(np.random.randint(1000000))
predict_result[['instance_id', 'predicted_score']].to_csv( "submission/%s_lgb_n%d_b%d_1h_col%d.csv" % (now, n_fold, 70, colnum), index=False)
np.save('submission/oof_%s_%.5f'%(now, cv_loss), oof) 